# clj-leaflet

An exploration of ipyleaflet compatible models for clojupyter kernel.

## Setup
To generate the models, this notebook requires a clojupyter kernel with ipywidgets support. Compile and install the latest version [feature/ipywidgets](https://github.com/clojupyter/clojupyter/tree/feature/ipywidgets) branch.

To view and interact with the models you need to install the jupyter lab/notebook extensions for [ipyleaflet](https://github.com/jupyter-widgets/ipyleaflet) and [ipywidgets](https://github.com/jupyter-widgets/ipywidgets).

In [1]:
(require '[clojupyter.widgets.ipywidgets :as ipy])
(require '[clojure.data.json :as json])
(require '[clojure.java.io :as io])
(require '[clojupyter.kernel.comm-atom :as ca])
(require '[clojupyter.state :as st])
(require '[clojupyter.util-actions :as u])
(require '[clojure.repl :refer :all])
(require '[clojure.pprint :refer [pprint]])
(require '[camel-snake-kebab.core :as csk]);

Hack to fix an incompatibility between latest version of jupyterlab manager and jupyter-leaflet.
In order to use an older version of jupyterlab manager we need to enforce the generated widget versions to an earlier version.

In [2]:
(defn old
  ([constr] (old constr {}))
  ([constr state] (constr (merge state {:_model_module_version "1.5.0" :_view_module_version "1.5.0"}))))
#_(defn old
  ([constr] (old constr {}))
  ([constr state] (constr (merge state {}))))

#'user/old

Helper macro useful to attach widgets to global vars and show them in a single command.

In [3]:
(defmacro def-show
  "Like def, but returns `init` if one was passed along."  
  ([symb]             `(def ~symb))
  ([symb init]         `(do (def ~symb ~init) ~init))
  ([symb docstr init] `(do (def ~symb ~docstr ~init) ~init)))

#'user/def-show

----
## Leaflet Factory

### Colors
Choropleth models in python use an extenal dependency to compute colors from values.
We define single-hue-linear-fn at the bottom of this section to achieve the same goal.

In [4]:
(defn abs
  [x]
  (if (neg? x) (- x) x))

#'user/abs

In [5]:
(defn str->rgb
  "Converts a string colour to a three int rbg tuple"
  [s]
  (->> (rest s)
       (partition 2)
       (map (partial reduce str))
       (map #(Integer/parseInt % 16))))

#'user/str->rgb

In [6]:
(defn rgb->hsv
  [clr]
  (let [[r g b] (map #(/ % 255) clr)
        c-max (max r g b)
        c-min (min r g b)
        dif (- c-max c-min)
        v c-max
        s (if (zero? dif) 0 (/ dif c-max))
        h (if (zero? dif)
            0
            (condp = c-max
              r (* 60 (mod (/ (- g b) dif) 6))
              g (* 60 (+ 2 (/ (- b r) dif)))
              b (* 60 (+ 4 (/ (- r g) dif)))))]
    (map float (list h s v))))

#'user/rgb->hsv

In [7]:
(defn hsv->rgb
  [[h s v]]
  {:pre [(<= 0 h 360)
         (<= 0 s 1)
         (<= 0 v 1)]}
  (let [c (* v s)
        q (/ h 60)
        x (* c (- 1 (abs (dec (mod q 2)))))
        [r1 g1 b1] (condp #(<= (first %1) %2 (second %1)) q
                     [0 1] [c x 0]
                     [1 2] [x c 0]
                     [2 3] [0 c x]
                     [3 4] [0 x c]
                     [4 5] [x 0 c]
                     [5 6] [c 0 x]
                     :else [0 0 0])
        m (- v c)]
    (map (comp int (partial * 255) (partial + m)) [r1 g1 b1])))

#'user/hsv->rgb

In [8]:
(defn rgb->str
  [[r g b]]
  {:pre [(every? int? [r g b])
         (every? #(<= 0 % 255) [r g b])]}
  (format "#%02X%02X%02X" r g b))

#'user/rgb->str

In [9]:
(defn single-hue-linear-fn
  ([v-min v-max] (single-hue-linear-fn v-min v-max "#BE481F"))
  ([v-min v-max color]
   (fn [x]
     {:pre [(<= v-min x v-max)]}
     (let [[h _ v] (-> color str->rgb rgb->hsv)]
       (-> (list h (float (/ (- x v-min) (- v-max v-min))) v)
           hsv->rgb
           rgb->str)))))

#'user/single-hue-linear-fn

#### Color transform test

In [10]:
(let [pc (old ipy/color-picker)
      lab (old ipy/label {:value (:value @pc)})
      _ (.watch pc :on-change-value (fn [_ _ {o-val :value} {n-val :value}]
                                      (when (not= o-val n-val)
                                        (swap! lab assoc :value (-> n-val str->rgb rgb->hsv pr-str)))))]
  (old ipy/v-box {:children [pc lab]}))  

[05eccf17-e5ab-46ea-80b3-e1ff4657a26b]={:_view_module "@jupyter-widgets/controls", :children [#<CommAtom@a61bace8:
  {:description "",
   :_view_module "@jupyter-widgets/controls",
   :disabled false,
   :layout nil,
   :value "black",
   :_view_module_version "1.5.0",
   :style nil,
   :tabbable nil,
   :_view_name "ColorPickerView",
   :_model_module "@jupyter-widgets/controls",
   :_model_name "ColorPickerModel",
   :concise false,
   :_dom_classes [],
   :tooltip nil,
   :_model_module_version "1.5.0"} #<CommAtom@4e30ec34:
  {:description "",
   :_view_module "@jupyter-widgets/controls",
   :placeholder "​",
   :layout nil,
   :value "black",
   :_view_module_version "1.5.0",
   :style nil,
   :tabbable nil,
   :_view_name "LabelView",
   :_model_module "@jupyter-widgets/controls",
   :_model_name "LabelModel",
   :_dom_classes [],
   :tooltip nil,
   :_model_module_version "1.5.0"}], :layout nil, :box_style "", :_view_module_version "1.5.0", :tabbable nil, :_view_name "VBoxView", 

### Models

In [11]:
(def SPECS (-> "resources/leaflet-schema.min.json"
               #_io/resource
               slurp
               json/read-str))

#'user/SPECS

In [12]:
(defn def-widget
  [{attributes "attributes"}]
  (let [all-attr (->> attributes (map #(get % "name")) (map keyword) (filterv (partial not= :layers)))]
    (reduce merge
      (for [{name "name" default "default" type "type"} attributes]
        {(keyword name) (cond
                          (= name "options") all-attr
                          (= type "reference") nil
                          :else default)}))))

#'user/def-widget

In [13]:
(defn make-widget
  [spec]
  (fn constructor
    [& args]
    (let [d-state (def-widget spec)
          state (merge d-state (apply hash-map args))
          target "jupyter.widget"
          {jup :jup req-msg :req-message} (st/current-context)
          id (u/uuid)
          v-keys (set (keys d-state))]
      (ca/create-and-insert jup req-msg target id v-keys state))))

#'user/make-widget

In [14]:
(for [{n "name" :as spec} SPECS]
  (eval `(def ~(if (= n "map") 'Map (symbol n)) ~(make-widget spec))))

(#'user/ant-path #'user/attribution-control #'user/awesome-icon #'user/choropleth #'user/circle #'user/circle-marker #'user/control #'user/draw-control #'user/feature-group #'user/full-screen-control #'user/geo-data #'user/geo-json #'user/heatmap #'user/icon #'user/image-overlay #'user/layer #'user/layer-group #'user/layers-control #'user/legend-control #'user/local-tile-layer #'user/Map #'user/map-style #'user/marker #'user/marker-cluster #'user/measure-control #'user/path #'user/polygon #'user/polyline #'user/popup #'user/raster-layer #'user/rectangle #'user/scale-control #'user/search-control #'user/split-map-control #'user/tile-layer #'user/ui-layer #'user/vector-layer #'user/vector-tile-layer #'user/video-overlay #'user/widget-control #'user/wms-layer #'user/zoom-control #'user/Layout #'user/DOMWidget)

---
## Model Tests

We import the base maps json file, flatten it and transform its keys to keywords to make it easier to work with.
The original python dict `{"open-street-map" {"mapnick" {"base-map dict"}}}` becomes `{:open-street-map-mapnick {"base-map dict"}}`

In [15]:
(def BASE-MAPS
  (let [base-maps (json/read-str (slurp "./resources/basemaps.json"))
        k-maps (for [[outer-key v] base-maps]
                 (if (contains? v "name")
                   {(csk/->kebab-case-keyword outer-key) (clojure.walk/keywordize-keys v)}
                   (reduce merge
                           (for [[inner-key vv] v]
                             {(csk/->kebab-case-keyword (str outer-key inner-key)) (clojure.walk/keywordize-keys vv)}))))]
    (reduce merge k-maps)))

#'user/BASE-MAPS

**Note:** The base-map hash-map :url must be formatted with a date string to generate the right value.

### Map Controls

In [16]:
(def-show M0
  (let [sel (old ipy/dropdown {:options (->> BASE-MAPS keys (map name) sort vec) :description "Base Map"})
        b-maps (reduce merge (for [[k v] BASE-MAPS] {k (apply tile-layer (-> v (update :url format "2020-05-01") list* flatten))}))
        w-ctrl (widget-control :widget sel :position "topright")
        bm (apply tile-layer (flatten (list* (:open-topo-map BASE-MAPS))))
        dm (Map :center [4 112] :zoom 3.6 :layers [bm] :scroll_wheel_zoom true
                :controls [(zoom-control) (attribution-control :position "bottomright") (scale-control :position "bottomleft") (full-screen-control) w-ctrl (search-control)])
        _ (.watch sel :on-select (fn [_ _ {o-val :value} {n-val :value}] (when (not= o-val n-val) (swap! dm update :layers (comp #(conj % (get b-maps (keyword n-val))) pop)))))]  
    dm))

[e4e9dbac-81ca-4019-a1a0-3ef947406337]={:tap_tolerance 15, :zoom_start 12, :tap true, :keyboard true, :_view_module "jupyter-leaflet", :window_url "", :zoom 3.6, :interpolation "bilinear", :west 0.0, :south 0.0, :layout nil, :scroll_wheel_zoom true, :bounce_at_zoom_limits true, :min_zoom 1, :crs "EPSG3857", :inertia true, :_view_module_version "^0.12.6", :controls [#<CommAtom@fd3e893d:
  {:_view_module "jupyter-leaflet",
   :_view_module_version "^0.12.6",
   :zoom_in_title "Zoom in",
   :zoom_out_text "-",
   :zoom_in_text "+",
   :position "topleft",
   :options
   [:_model_module
    :_model_module_version
    :_model_name
    :_view_module
    :_view_module_version
    :_view_name
    :options
    :position
    :zoom_in_text
    :zoom_in_title
    :zoom_out_text
    :zoom_out_title],
   :_view_name "LeafletZoomControlView",
   :_model_module "jupyter-leaflet",
   :_model_name "LeafletZoomControlModel",
   :zoom_out_title "Zoom out",
   :_model_module_version "^0.12.6"} #<CommAtom@3

In [17]:
(def-show M1
  (let [l0 (apply tile-layer (flatten (apply list (:esri-world-imagery BASE-MAPS))))
        l1 (apply tile-layer (flatten (apply list (:nasagibs-viirs-earth-at-night-2012 BASE-MAPS))))
        sm (split-map-control :left_layer l0 :right_layer l1)
        m (Map :controls [sm] :center [46, -35] :zoom 3)]
    m))

[5dc6d416-f4a1-44ad-b04d-07b7f956d4ca]={:tap_tolerance 15, :zoom_start 12, :tap true, :keyboard true, :_view_module "jupyter-leaflet", :window_url "", :zoom 3, :interpolation "bilinear", :west 0.0, :south 0.0, :layout nil, :scroll_wheel_zoom false, :bounce_at_zoom_limits true, :min_zoom 1, :crs "EPSG3857", :inertia true, :_view_module_version "^0.12.6", :controls [#<CommAtom@52436326:
  {:_view_module "jupyter-leaflet",
   :_view_module_version "^0.12.6",
   :left_layer
   #<CommAtom@4005b378:
     {:_view_module "jupyter-leaflet",
      :popup nil,
      :popup_max_width 300,
      :popup_max_height nil,
      :bottom true,
      :name "Esri.WorldImagery",
      :min_zoom 0,
      :popup_min_width 50,
      :_view_module_version "^0.12.6",
      :detect_retina false,
      :min_native_zoom 0,
      :no_wrap false,
      :loading false,
      :opacity 1.0,
      :attribution
      "Tiles &copy; Esri &mdash; Source: Esri, i-cubed, USDA, USGS, AEX, GeoEye, Getmapping, Aerogrid, IGN, IGP,

In [18]:
(let [bm (apply tile-layer (flatten (apply list (:carto-db-dark-matter BASE-MAPS))))
      dc (draw-control)
      mc (measure-control :primary_length_unit "kilometers" :primary_area_unit "hectares")]
  (Map :center [38 27] :zoom 6 :layers [bm] :controls [dc mc]))

[756e58c6-9bee-4af1-9411-10a57b994d7a]={:tap_tolerance 15, :zoom_start 12, :tap true, :keyboard true, :_view_module "jupyter-leaflet", :window_url "", :zoom 6, :interpolation "bilinear", :west 0.0, :south 0.0, :layout nil, :scroll_wheel_zoom false, :bounce_at_zoom_limits true, :min_zoom 1, :crs "EPSG3857", :inertia true, :_view_module_version "^0.12.6", :controls [#<CommAtom@ecb72bf8:
  {:_view_module "jupyter-leaflet",
   :remove true,
   :circlemarker {"shapeOptions" {}},
   :marker {},
   :edit true,
   :circle {},
   :_view_module_version "^0.12.6",
   :position "topleft",
   :options
   [:_model_module
    :_model_module_version
    :_model_name
    :_view_module
    :_view_module_version
    :_view_name
    :circle
    :circlemarker
    :edit
    :marker
    :options
    :polygon
    :polyline
    :position
    :rectangle
    :remove],
   :_view_name "LeafletDrawControlView",
   :_model_module "jupyter-leaflet",
   :polyline {"shapeOptions" {}},
   :_model_name "LeafletDrawContro

**TODO:** search-control example.

### Markers

Awesome-icon widget is not dynamic. You must refer to a new one to change it.

In [19]:
(def-show M2
  (let [loc [52.205 360.121]
        lay (tile-layer)
        ico (awesome-icon :name "address-card" :title ":boo")
        pin (marker :location loc :draggable true :rise_on_hover true :icon ico)
        lab (old ipy/label {:value (str "Pin at " (:location @pin))})
        m (Map :center loc :zoom 15 :layers [lay pin])
        text (old ipy/text {:description "Icon Tooltip"})
        _ (.watch text :on-change-value
             (fn [_ _ {o-val :value} {n-val :value}]
               (when (not= o-val n-val)
                 (swap! ico assoc :name n-val))))
        boolin (old ipy/checkbox {:description "Spin?"})
        l0 (old ipy/directional-link {:source [boolin :value] :target [ico :spin]})
        l1 (old ipy/directional-link {:source [text :value] :target [pin :title]})
        _ (.watch pin :on-change-location (fn [_ _ {o-loc :location} {n-loc :location}]
                                           (when (not= o-loc n-loc)
                                             (swap! lab assoc :value (str "Pin at " (reduce conj [] (map (partial format "%.4f") n-loc)))))))
        in-b (old ipy/h-box {:children [text boolin]})]
  (old ipy/v-box {:children [in-b m lab]})))

[15e475b4-032f-4b07-9d8a-0b0cd80d1737]={:_view_module "@jupyter-widgets/controls", :children [#<CommAtom@2e224e59:
  {:_view_module "@jupyter-widgets/controls",
   :children
   [#<CommAtom@7cb987b9:
      {:description "Icon Tooltip",
       :_view_module "@jupyter-widgets/controls",
       :placeholder "​",
       :disabled false,
       :layout nil,
       :value "",
       :_view_module_version "1.5.0",
       :continuous_update true,
       :style nil,
       :tabbable nil,
       :_view_name "TextView",
       :_model_module "@jupyter-widgets/controls",
       :_model_name "TextModel",
       :_dom_classes [],
       :tooltip nil,
       :_model_module_version "1.5.0"}>
    #<CommAtom@7bced7fa:
      {:description "Spin?",
       :_view_module "@jupyter-widgets/controls",
       :indent true,
       :disabled false,
       :layout nil,
       :value false,
       :_view_module_version "1.5.0",
       :style nil,
       :tabbable nil,
       :_view_name "CheckboxView",
       :_mod

### Paths
#### Ant-Path

In [20]:
(def-show M4
  (let [ap [(ant-path :locations [[51.5 8] [52.5 12.1]] :use "polyline")
            (ant-path :locations [[52 8] [52.5 8] [52.3 9]] :use "polygon" :dash-array [50 100] :weight 10 :color "red")
            (ant-path :locations [[51.5 11] [52 12]] :use "rectangle" :dash_array [10 20] :weight 5 :color "white" :pulse_color "green")            
            (ant-path :locations [51.5 10] :radius 15000 :use "circle" :dash_array [5 10] :weight 5 :color "magenta")]]
    (Map :center [52 10] :zoom 8 :layers (into [(tile-layer)] ap) :controls [(zoom-control) (attribution-control)])))

[f9625571-34d7-4038-a851-35b70f124236]={:tap_tolerance 15, :zoom_start 12, :tap true, :keyboard true, :_view_module "jupyter-leaflet", :window_url "", :zoom 8, :interpolation "bilinear", :west 0.0, :south 0.0, :layout nil, :scroll_wheel_zoom false, :bounce_at_zoom_limits true, :min_zoom 1, :crs "EPSG3857", :inertia true, :_view_module_version "^0.12.6", :controls [#<CommAtom@5ad524fd:
  {:_view_module "jupyter-leaflet",
   :_view_module_version "^0.12.6",
   :zoom_in_title "Zoom in",
   :zoom_out_text "-",
   :zoom_in_text "+",
   :position "topleft",
   :options
   [:_model_module
    :_model_module_version
    :_model_name
    :_view_module
    :_view_module_version
    :_view_name
    :options
    :position
    :zoom_in_text
    :zoom_in_title
    :zoom_out_text
    :zoom_out_title],
   :_view_name "LeafletZoomControlView",
   :_model_module "jupyter-leaflet",
   :_model_name "LeafletZoomControlModel",
   :zoom_out_title "Zoom out",
   :_model_module_version "^0.12.6"} #<CommAtom@cd

#### Polylines

In [21]:
(let [p0 (polyline :locations [[45.51 -122.68] [37.77 -122.43] [34.04 -118.22]] :color "green" :fill false)
      p1 (polyline :locations [[[45.51 -122.68] [37.77 -122.43] [34.04 -118.22]]
                              [[40.78 -73.91] [41.83 -87.62] [32.76 -96.72]]]
                  :color "green" :fill false)
      p2 (polygon :locations [[42 -49] [43 -49] [43 -48]] :color "green" :fill_color "green")
      p3 (polygon :locations [[[37 -109.05] [41 -109.03] [41 -102.05] [37 -102.04]]
                             [[37.29 -108.58] [40.71 -108.58] [40.71 -102.50] [37.29 -102.50]]]
                 :color "green" :fill_color "green")
      p4 (rectangle :bounds [[52 354] [53 360]])
      p5 (circle :location [50 354] :radius 50 :color "green" :fill_color "green")
      p6 (circle-marker :location [55 360] :radius 50 :color "red" :fill_color "red")
      b-in (old ipy/toggle-buttons {:options ["Polyline" "Multi-Polyline" "Polygon" "Multi-Polygon" "Rectangle" "Circle" "Circle-Marker"]})
      paths [p0 p1 p2 p3 p4 p5 p6]
      switch-lay (fn [lays n-idx] (-> lays pop (conj (get paths n-idx))))
      m (Map :center [42.5 -41] :zoom 2 :layers [(tile-layer) p0])
      _ (.watch b-in :on-change-index (fn [_ _ {o-idx :index} {n-idx :index}]
                                        (when (not= o-idx n-idx)
                                          (swap! m update :layers switch-lay n-idx))))]
  (old ipy/v-box {:children [b-in m]}))

[2c45a4d4-9d1c-4f74-864e-65b2364aefe3]={:_view_module "@jupyter-widgets/controls", :children [#<CommAtom@182f7386:
  {:description "",
   :_view_module "@jupyter-widgets/controls",
   :index nil,
   :tooltips [],
   :disabled false,
   :button_style "",
   :layout nil,
   :_options_labels
   ["Polyline"
    "Multi-Polyline"
    "Polygon"
    "Multi-Polygon"
    "Rectangle"
    "Circle"
    "Circle-Marker"],
   :_view_module_version "1.5.0",
   :icons [],
   :style nil,
   :tabbable nil,
   :options
   ["Polyline"
    "Multi-Polyline"
    "Polygon"
    "Multi-Polygon"
    "Rectangle"
    "Circle"
    "Circle-Marker"],
   :_view_name "ToggleButtonsView",
   :_model_module "@jupyter-widgets/controls",
   :_model_name "ToggleButtonsModel",
   :_dom_classes [],
   :_options_values
   ["Polyline"
    "Multi-Polyline"
    "Polygon"
    "Multi-Polygon"
    "Rectangle"
    "Circle"
    "Circle-Marker"],
   :tooltip nil,
   :_model_module_version "1.5.0"} #<CommAtom@cb2c50c6:
  {:tap_tolerance 1

### Audio / Video Overlay

In [22]:
(let [v (video-overlay :url "https://www.mapbox.com/bites/00188/patricia_nasa.webm" :bounds [[13 -130] [32 -100]])
      m (Map :center [23 -115] :zoom 4 :layers [(tile-layer) v])]
  m)

[b65201f6-fe7e-4ec3-ba99-56d440071df5]={:tap_tolerance 15, :zoom_start 12, :tap true, :keyboard true, :_view_module "jupyter-leaflet", :window_url "", :zoom 4, :interpolation "bilinear", :west 0.0, :south 0.0, :layout nil, :scroll_wheel_zoom false, :bounce_at_zoom_limits true, :min_zoom 1, :crs "EPSG3857", :inertia true, :_view_module_version "^0.12.6", :controls [], :default_style nil, :world_copy_jump false, :dragging true, :east 0.0, :touch_zoom true, :double_click_zoom true, :zoom_animation_threshold 4, :center [23 -115], :style nil, :close_popup_on_click true, :modisdate "yesterday", :inertia_deceleration 3000, :keyboard_zoom_offset 1, :fullscreen false, :inertia_max_speed 1500, :tabbable nil, :max_zoom 18, :dragging_style nil, :keyboard_pan_offset 80, :layers [#<CommAtom@e46aed2a:
  {:_view_module "jupyter-leaflet",
   :popup nil,
   :popup_max_width 300,
   :popup_max_height nil,
   :bottom true,
   :name "",
   :min_zoom 0,
   :popup_min_width 50,
   :_view_module_version "^0.1

---
## Geo Data

In [23]:
(def GEO-DATA (clojure.walk/keywordize-keys (json/read-str (slurp "./resources/data/world_bank/WB_countries_Admin0_lowres.geo.json"))))
(def POP-MAP (reduce merge (map #(hash-map (:ISO_A3 (:properties %)) (:POP_EST (:properties %))) (:features GEO-DATA))));

Geo Data example uses pandas to load a dataset into memory. We need to identify its serial form to replicate on clojure side.

### Geo-JSON

In [24]:
(def-show M3
  (let [lab (old ipy/label)
        l0 (apply tile-layer (flatten (list* (:open-street-map-black-and-white BASE-MAPS))))
        d (->> (:features GEO-DATA)
               (map #(update % :properties assoc :style {:color "grey" :weight 1 :fillColor "lightyellow" :fillOpacity 0.5})))
        h (fn [_ msg _] (swap! lab assoc :value (get-in msg [:properties :WB_NAME] "")))
        l1 (geo-json :data d :hover_style {:fillColor "red"} :callbacks {:on-mouseover h})
        m (Map :center [53.88 27.45] :zoom 4 :layers [l0 l1] :tooltip lab)]
  (old ipy/v-box {:children [m lab]})))

[df48cace-2227-49e3-8816-97199871d1d3]={:_view_module "@jupyter-widgets/controls", :children [#<CommAtom@50a7b83a:
  {:tap_tolerance 15,
   :zoom_start 12,
   :tap true,
   :keyboard true,
   :_view_module "jupyter-leaflet",
   :window_url "",
   :zoom 4,
   :interpolation "bilinear",
   :west 0.0,
   :south 0.0,
   :layout nil,
   :scroll_wheel_zoom false,
   :bounce_at_zoom_limits true,
   :min_zoom 1,
   :crs "EPSG3857",
   :inertia true,
   :_view_module_version "^0.12.6",
   :controls [],
   :default_style nil,
   :world_copy_jump false,
   :dragging true,
   :east 0.0,
   :touch_zoom true,
   :double_click_zoom true,
   :zoom_animation_threshold 4,
   :center [53.88 27.45],
   :style nil,
   :close_popup_on_click true,
   :modisdate "yesterday",
   :inertia_deceleration 3000,
   :keyboard_zoom_offset 1,
   :fullscreen false,
   :inertia_max_speed 1500,
   :tabbable nil,
   :max_zoom 18,
   :dragging_style nil,
   :keyboard_pan_offset 80,
   :layers
   [#<CommAtom@80f26829:
      

In [25]:
(:cursor @(map-style))

"grab"

In [26]:
(apply distinct?  (repeat 5 (old ipy/int-slider)))

false

### Choropleth
Choropleth model is a geo-json with custom coloring based on some data.
Python model has external dependency for choosing the colours.

In [27]:
(let [v-max (reduce max (vals POP-MAP))
      v-min (reduce min (vals POP-MAP))
      col-fn (single-hue-linear-fn v-min v-max "#D61717")
      lab (old ipy/label)
      g-data (update GEO-DATA :features #(map (fn [m] (assoc-in m [:properties :style] {:weight 0.9 :color "black" :fillColor (col-fn (get POP-MAP (get-in m [:properties :ISO_A3]) v-min))})) %))
      c (choropleth :data g-data :style {:fillOpacity 0.8 :dashArray "5, 5"} :callbacks {:on-mouseover (fn [_ {{name :WB_NAME id :ISO_A3} :properties} _] (swap! lab assoc :value (format "Population %s: %,d" name (get POP-MAP id))))})
      leg-k (take 8 (iterate (partial + (quot (- v-max v-min) 7)) v-min))
      leg (legend-control :position "bottomleft" :legend (zipmap leg-k (map col-fn leg-k)))
      m (Map :layers [(tile-layer) c] :center [45 90] :zoom 3 :scroll_wheel_zoom true :controls [(full-screen-control) (widget-control :widget lab :position "topright") leg])]
  m)  

[e51bf7ed-7e67-4927-ae11-e93b3a7644b9]={:tap_tolerance 15, :zoom_start 12, :tap true, :keyboard true, :_view_module "jupyter-leaflet", :window_url "", :zoom 3, :interpolation "bilinear", :west 0.0, :south 0.0, :layout nil, :scroll_wheel_zoom true, :bounce_at_zoom_limits true, :min_zoom 1, :crs "EPSG3857", :inertia true, :_view_module_version "^0.12.6", :controls [#<CommAtom@a208c94f:
  {:_model_module "jupyter-leaflet",
   :_model_module_version "^0.12.6",
   :_model_name "LeafletFullScreenControlModel",
   :_view_module "jupyter-leaflet",
   :_view_module_version "^0.12.6",
   :_view_name "LeafletFullScreenControlView",
   :options
   [:_model_module
    :_model_module_version
    :_model_name
    :_view_module
    :_view_module_version
    :_view_name
    :options
    :position],
   :position "topleft"} #<CommAtom@da64b0c1:
  {:_view_module "jupyter-leaflet",
   :min_height nil,
   :_view_module_version "^0.12.6",
   :max_width nil,
   :widget
   #<CommAtom@45b8b40c:
     {:descripti

### Heatmap

In [28]:
(let [loc (vec (repeatedly 1000 #(vector (- (rand 160) 80) (- (rand 360) 180) (rand 1000))))
      hm (heatmap :locations loc :radius 20 :opacity 0.9)
      m (Map :center [0 0] :zoom 2 :layers [(tile-layer) hm])]
 m)

[23074855-3ed7-41f5-bfe5-356ed32474b6]={:tap_tolerance 15, :zoom_start 12, :tap true, :keyboard true, :_view_module "jupyter-leaflet", :window_url "", :zoom 2, :interpolation "bilinear", :west 0.0, :south 0.0, :layout nil, :scroll_wheel_zoom false, :bounce_at_zoom_limits true, :min_zoom 1, :crs "EPSG3857", :inertia true, :_view_module_version "^0.12.6", :controls [], :default_style nil, :world_copy_jump false, :dragging true, :east 0.0, :touch_zoom true, :double_click_zoom true, :zoom_animation_threshold 4, :center [0 0], :style nil, :close_popup_on_click true, :modisdate "yesterday", :inertia_deceleration 3000, :keyboard_zoom_offset 1, :fullscreen false, :inertia_max_speed 1500, :tabbable nil, :max_zoom 18, :dragging_style nil, :keyboard_pan_offset 80, :layers [#<CommAtom@f21c5bb0:
  {:_view_module "jupyter-leaflet",
   :popup nil,
   :popup_max_width 300,
   :popup_max_height nil,
   :bottom true,
   :name "",
   :min_zoom 0,
   :popup_min_width 50,
   :_view_module_version "^0.12.6"

### Velocity
**TODO:**

---
## Conclusions

### Layer Groups / Marker Clusters

Do we need them? Can we use a simple collection of layers/markers?

### Abstract Classes

control, feature-group, layer, layers-control, path, raster-layer, ui-layer, vector-layer, vector-tile-layer, Layout and dom-widgets look like abstract classes. Does it make sense to keep them on clojure side?

----
## Debug

In [29]:
#_(taoensso.timbre/set-level! :debug)

In [30]:
#_(let [p (polyline :locations [[[45.51 -122.68] [37.77 -122.43] [34.04 -118.22]]
                              [[40.78 -73.91] [41.83 -87.62] [32.76 -96.72]]]
                  :color "green" :fill false)
      m (Map :center [42.5 -41] :zoom 2 :options [:center :zoom] :layers [(tile-layer) p])]
  m)

In [31]:
#_(let [p (polygon :locations [[42 -49] [43 -49] [43 -48]] :color "green" :fill_color "green")
      m (Map :center [42.5531 -48.6914] :zoom 6 :options [:center :zoom] :layers [(tile-layer) p])]
  m)

In [32]:
#_(let [p (polygon :locations [[[37 -109.05] [41 -109.03] [41 -102.05] [37 -102.04]]
                             [[37.29 -108.58] [40.71 -108.58] [40.71 -102.50] [37.29 -102.50]]]
                 :color "green" :fill_color "green")
      m (Map :center [37.5531 -109.6914] :zoom 5 :options [:center :zoom] :layers [(tile-layer) p])]
  m)

In [33]:
#_(let [pr (rectangle :bounds [[52 354] [53 360]])
      pc (circle :location [50 354] :radius 50 :color "green" :fill_color "green")
      m (Map :center [53 354] :zoom 5 :options [:center :zoom] :layers [(tile-layer) pr pc])]
  m)

In [34]:
#_(let [cm (circle-marker :location [55 360] :radius 50 :color "red" :fill_color "red")
      m (Map :center [53 354] :zoom 5 :options [:center :zoom] :layers [(tile-layer) cm])]
  m)

In [35]:
#_(->> [(ant-path :locations [[51.5 8] [52.5 12.1]] :use "polyline" :options [:use])
      (ant-path :locations [[52 8] [52.5 8] [52.3 9]] :use "polygon" :dash-array [50 100] :weight 10 :color "red")
      (ant-path :locations [[51.5 11] [52 12]] :use "rectangle" :dash_array [10 20] :weight 5 :color "white" :pulse_color "green")
      (ant-path :locations [51.5 10] :radius 15000 :use "circle" :dash_array [5 10] :weight 5 :color "magenta")]
     last
     (swap! M3 update :layers conj))

In [36]:
#_(clojure.pprint/pprint
 (filter string?
   (for [{n "name" att "attributes"} SPECS
         {a-name "name" a-type "type" nilable? "allow_none"} att]
     (when (= a-type "reference")
       (str "Reference found on widget " n ", attribute " a-name ", nilable? " nilable?)))))